## Reading temperature directly from the TemperaSure Application

In [23]:
from pywinauto.application import Application

app = Application(backend='uia').start(r"C:\Users\Lab10\Desktop\TemperaSure.exe").connect(title = r'BASF TemperaSure 5.7.0.4 Advanced Mode')
# .start(r"C:\Users\Lab10\Desktop\TemperaSure.exe")
dlg = app.window(title = r'BASF TemperaSure 5.7.0.4 Advanced Mode')

# Print all controls recursively
# dlg.print_control_identifiers()
# Find the toolbar containing the temperature (ToolBar2)
toolbar2 = dlg.child_window(control_type="ToolBar", title_re="ToolBar2")

# Find all Edit controls inside this toolbar
edit_controls = toolbar2.children(control_type="Edit")

# If there’s only one Edit control, assume it’s the temperature
if edit_controls:
    temp_edit = edit_controls[0]
    temperature = temp_edit.get_value()
    print("Temperature:", temperature)
else:
    print("No Edit control found in ToolBar2")


Temperature: 165.00


## Start pyrometer

In [13]:
from pywinauto.application import Application
import time

try:
    app = Application(backend='uia').connect(title = r'BASF TemperaSure 5.7.0.4 Advanced Mode')
    # time.sleep(1)
    app.kill()
except:
    print('BASF Pyrometer software currently not running')

print('starting BASF pyrometer software')
app = Application(backend='uia').start(r"C:\Users\Lab10\Desktop\TemperaSure.exe")
time.sleep(6)
main = app.window(title = r'BASF TemperaSure 5.7.0.4 Advanced Mode')
# dlg = app.window(title="Port setup") 
# main.print_control_identifiers()
port = main.child_window(title="Port setup", control_type="Window")
# dlg = app.window(title="Port setup")  
# dlg.wait("visible enabled", timeout=2)

# # Step 2: Click the OK button
# # Most dialogs have a button literally called "OK"
# dlg["OK"].click()

try:
    port.set_focus()
    port.type_keys("{ENTER}")
    print('clicked the ok button')
    time.sleep(5)
except Exception:
    print('was not able to click the ok button')
    pass

# Try to click start button
# main.print_control_identifiers()
try:
    main.child_window(title="Start", control_type="Button").click_input()
    print('clicked the start button')
except:
    print('was unable to click the start button')


starting BASF pyrometer software
clicked the ok button
clicked the start button


In [ ]:
from time import sleep
from pywinauto import Application, Desktop
from pywinauto.timings import TimeoutError as PywinautoTimeoutError

EXE = r"C:\Users\Lab10\Desktop\TemperaSure.exe"
MAIN_TITLE = "BASF TemperaSure 5.7.0.4 Advanced Mode"

def click_ok_on_port_popup(exe_path=EXE, backend="uia"):
    # 1) Start the app (backend A)
    app = Application(backend=backend).start(exe_path)

    # Optional: give the app a moment to spin up
    try:
        app.wait_cpu_usage_lower(threshold=5, timeout=10)
    except Exception:
        pass

    pid = app.process

    # Helper to try clicking OK on any "Port" dialog for a given backend
    def _try_backend(bk: str) -> bool:
        dsk = Desktop(backend=bk)

        # Try a few common title patterns
        title_patterns = [r".*Port.*", r".*Port setup.*", r".*Select.*Port.*", r".*COM.*"]
        for pat in title_patterns:
            try:
                dlg = dsk.window(title_re=pat, process=pid, control_type="Window")
                dlg.wait("exists visible enabled", timeout=8)
                # Try the obvious OK button
                try:
                    dlg.child_window(title="OK", control_type="Button").wrapper_object().click_input()
                    return True
                except Exception:
                    pass
                # Fallbacks: first button, or press Enter
                try:
                    ok_btn = dlg.child_window(control_type="Button", found_index=0).wrapper_object()
                    ok_btn.click_input()
                    return True
                except Exception:
                    pass

                try:
                    dlg.set_focus()
                    dlg.type_keys("{ENTER}")
                    return True
                except Exception:
                    pass
            except PywinautoTimeoutError:
                continue
            except Exception:
                continue
        return False

    # 2) Try UIA first
    if not _try_backend("uia"):
        # 3) If that didn’t work, try Win32 (some dialogs only expose Win32)
        _try_backend("win32")

    # 4) Attach to main window (optional)
    try:
        win = app.window(title=MAIN_TITLE)
        win.wait("visible enabled", timeout=20)
    except Exception:
        pass

    return app

# --- usage ---
app = click_ok_on_port_popup()